## 데이터 인코딩

머신러닝을 위한 대표적인 인코딩 방식은 `레이블 인코딩(Label encoding)`과 `원-핫 인코딩(One Hot encoding)`이 있다.

`레이블 인코딩` 은 카테고리 피처를 코드형 숫자 값으로 변환하는 것이다.

<br>

`원 핫 인코딩`은 피처 값의 유형에 따라 새로운 피처를 추가해 고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시한다.

### 레이블 인코딩

사이킷런의 레이블 인코딩(Label encoding)은 LabelEncoder 클래스로 구현합니다. LabelEncoder를 객체로 생성한 후 fit()과 transform을 호출해 레이블 인코딩을 수행한다.

In [1]:
from sklearn.preprocessing import LabelEncoder

In [2]:
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

In [3]:
# LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 레이블 인코딩 수행.
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:', labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [4]:
print('인코딩 클래스:', encoder.classes_)

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [5]:
print('디코딩 원본값:', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

디코딩 원본값: ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


레이블 인코딩은 간단하게 문자열 값을 숫자형 카테고리 값으로 변환한다.

하지만 레이블 인코딩이 일괄적인 숫자 값으로 변환이 되면서 몇몇 ML 알고리즘에는 이를 적용할 경우 예측 성능이 떨어지는 경우가 발생할 수 있음.

이는 숫자 값의 경우 크고 작음에 대한 특성이 작용하기 때문 -> *가중치 발생*

이러한 특성 때문에 레이블 인코딩은 `선형 회귀`와 같은 ML 알고리즘에는 적용하지 않아야 한다.

`트리 계열`의 ML 알고리즘은 이러한 숫자의 특성을 반영하지 않으므로 레이블 인코딩도 문제 없다.

---

<br>

`원 핫 인코딩(One-Hot Encoding)`은 레이블 인코딩의 이러한 문제점을 해결하기 위한 인코딩 방식

*원-핫(여러 개의 속성 중 단 한개의 속성만 1로 표시)*

원-핫 인코딩은 사이킷런에서 OneHotEncoder 클래스로 쉽게 변환 가능.

단, LabelEncoder와 다르게 주의할 점이 있는데   
1. OneHotEncder로 변환하기 전에 모든 문자열 값이 숫자로 변환돼야 함
2. 입력 값으로 2차원 데이터 필요

In [6]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [7]:
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

In [8]:
# 먼저 숫자 값으로 변환을 위해 LabelEncoder로 변환
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

In [9]:
# 2차원 데이터로 변환
labels = labels.reshape(-1, 1)

In [10]:
# 원-핫 인코딩 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

In [11]:
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)


판다스에 원-핫 인코딩을 더 쉽게 지원하는 API가 있음.

get_dummies()를 이용하면 된다.

사이킷런의 OneHotEncoder와는 다르게 문자열 카테고리 값을 숫자 형으로 변환할 필요 X

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame({'item':['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})

pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


---

## 피처 스케일링과 정규화

서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업을 `피처 스케일링(feature scaling)`이라고 한다.

대표적인 방법으로는 `표준화(Standardization)`와 `정규화(Normalization)가 있다.

표준화는 데이터의 피처 각각이 평균이 0이고 분산이 1인 가우시안 정규 분포를 가진 값으로 변환하는 것을 의미.

표준화를 통해 변환될 피처 x의 새로운 i번째 데이터를 xi_new고 한다면

이 값은 원래 값에서 피처 x의 평균을 뺀 값을 피처 x의 표준편차로 나눈 값으로 계산할 수 있다.

<br>

일반적으로 `정규화` 는 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해주는 개념이다.

<br>

예를 들어 피처 A는 거리를 나타내는 변수로서 값이 0 ~ 100KM로 주어지고   
피처 B는 금액을 나타내는 속성으로 값이 0 ~ 100,000,000,000원으로 주어진다면

이 변수를 동일한 크기 단위로 비교하기 위해 값을 모두 최소 0 ~ 최대 1의 값으로 변환하는 것.

> *즉, 개별 데이터의 크기를 모두 똑같은 단위로 변경하는 것이다.*



---
### StandardScaler

StandardScaler는 앞서 설명한 표준화를 쉽게 지원하기 위한 클래스이다. 즉, 개별 피처를 평균이 0이고, 분산이 1인 값으로 변환해준다.

이렇게 가우시안 정규 분포를 가질 수 있도록 데이터를 변환하는 것은 몇몇 알고리즘에서 매우 중요하다.

특히 사이킷런에서 구현한 RBF 커널을 이용하는 서포트 벡터 머신(Support Vetor Machine)이나 선형 회귀(Linear Regression), 로지스틱 회귀(Logistic Regression)는 데이터가 가우시안 분포를 가지고 있다고 가정하고 구현됐기 때문에   
사전에 표준화를 적용하는 것은 예측 성능 향상에 중요한 요소가 될 수 있다.

In [14]:
from sklearn.datasets import load_iris
import pandas as pd

In [16]:
# 붓꽃 데이터 세트를 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

In [17]:
print('feature 들의 평균 값')
print(iris_df.mean())
print('\nfeature들의 분산 값')
print(iris_df.var())

feature 들의 평균 값
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature들의 분산 값
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


`StandardScaler` 를 이용해 각 피처를 한번에 표준화해 변환한다.

StandardScaler 객체를 생성한 후에 fit()과 transform() 메서드에 변환 대상 피처 데이터 세트를 입력하고 호출하면 간단하게 변환 됨.

transform()을 호출할 때 스케일 변환된 데이터 세트가 넘파이의 ndarray이므로 이를 DataFrame으로 변환해 평균값과 분산 값을 다시 확인한다.

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
# StandardSclaer 객체 생성
scaler = StandardScaler()

In [20]:
# StandardScaler로 데이터 세트 변환. fit()과 transform() 호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

In [21]:
# transform() 시 스케일 변환된 데이터 세트가 NumPy ndarray로 반환돼 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

In [22]:
print('feature 들의 평균 값')
print(iris_df_scaled.mean())
print('feature 들의 분산 값')
print(iris_df_scaled.var())

feature 들의 평균 값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
feature 들의 분산 값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


모든 칼럼 값의 평균이 0에 아주 가까운 값으로,

그리고 분산은 1에 아주 가까운 갑으로 변환됐음을 알 수 있음.

---
### MinMaxScaler

`MinMaxScaler`는 데이터값을 0과 1사이의 범위 값으로 변환한다(음수 값이 있으면 -1에서 1 값으로 변환한다.)

데이터의 분포가 가우시안 분포가 아닐 경우에 Min, Max Scale을 적용해 볼 수 있다.


In [23]:
from sklearn.preprocessing import MinMaxScaler

In [25]:
# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# MinMaxScaler로 데이터 세트 변환. fit()과 transform() 호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

In [26]:
# transform() 시 스케일 변환된 데이터 세트가 Numpy ndarray로 반환돼 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

In [27]:
print('feature들의 최솟값')
print(iris_df_scaled.min())
print('feature들의 최댓값')
print(iris_df_scaled.max())

feature들의 최솟값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64
feature들의 최댓값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


모든 피처에 0에서 1 값으로 변환되는 스케일링이 적용됐음을 볼 수 있음.

---

## 학습 데이터와 테스트 데이터의 스케일링 변환 시 유의점

StandardScaler나 MinMaxScaler와 같은 Scaler 객체를 이용해 데이터 스케일링 변환 시 fit(), transform(), fit_transform() 메소드를 이용한다.

일반적으로 fit()은 데이터 변환을 위한 기준 정보 설정(예를 들어 데이터 세트의 최댓값/최솟값 설정 등)을 적용하며   
transform()은 이렇게 설정한 정보를 이용해 데이터를 변환한다.

그리고 fit_transform()은 fit()과 transform()을 한번에 적용하는 기능을 수행한다.

그런데 학습 데이터 세트와 테스트 데이터 세트에 이 fit()과 transform()을 적용할 때 주의가 필요하다.   
Scaler 객체를 이용해 학습 데이터 세트로 fit()과 transform()을 적용하면 테스트 데이터 세트로는 다시는 fit()을 수행하지 않고 학습 데이터 세트로 fit()을 수행한 결과를 이용해 transform() 변환을 적용해야 한다는 것이다.

In [30]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [32]:
# 학습 데이터는 0부터 10까지, 테스트 데이터는 0부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler 클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1, 1)로 차원 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [33]:
# MinMaxScaler 객체에 별도의 feature_range 파라미터 값을 지정하지 않으면 0~1 값으로 변환
scaler = MinMaxScaler()

# fit() 하게 되면 train_array 데이터의 최솟값이 0, 최댓값이 10으로 설정.
scaler.fit(train_array)

# 1/10 scale로 train_array 데이터 변환함. 원본 10->1 로 변환됨.
train_scaled = scaler.transform(train_array)

In [34]:
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale 된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale 된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [35]:
# MinMaxScaler에 test_array를 fit()하게 되면 원본 데이터의 최솟값이 0, 최댓값이 5로 설정됨
scaler.fit(test_array)

# 1/5 scale로 test_array를 변환함. 원본 5->1로 변환.
test_scaled = scaler.transform(test_array)

# test_array의 scale 변환 출력.
print('원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale 된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터: [0 1 2 3 4 5]
Scale 된 test_array 데이터: [0.  0.2 0.4 0.6 0.8 1. ]


출력 결과에서 학습 데이터와 테스트 데이터의 스케일링이 맞지 않음을 알 수 있다.

테스트 데이터의 경우는 최솟값 0, 최댓값 5이므로 1/5로 스케일링 된다.

따라서 원본값 1은 0.2로, 원본값 5는 1로 변환이 된다.

다음 코드는 테스트 데이터에 fit()을 호출하지 않고 학습 데이터로 fit()을 수행한 MinMaxScaler 객체의 transform()을 이용해 데이터를 변환한다.   
출력 결과를 확인해 보면 학습 데이터, 테스트 데이터, 모두 1/10 수준으로 스케일링 되어 1이 0.1로, 5가 0.5로, 학습 데이터, 테스트 데이터 모두 동일하게 변환됐음을 확인할 수 있다.

In [38]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)

In [41]:
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale 된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

# test_array에 Scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform()만으로 변환해야 함.
test_scaled = scaler.transform(test_array)
print('\n원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale 된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale 된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

원본 test_array 데이터: [0 1 2 3 4 5]
Scale 된 test_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5]


fit_transform()을 적용할 때도 마찬가지.
fit_transform()은 fit()과 transform을 순차적으로 수행하는 메소드이므로 학습 데이터에서는 상관없지만

테스트 데이터에서는 절대 사용해서는 안된다.

이렇게 학습과 테스트 데이터에 fit()과 transform()을 적용할 때 주의사항이 발생하므로 학습과 테스트 데이터 세트로 분리하기 전에 먼저 전체 데이터 세트에 스케일링을 적용한 뒤   
학습과 테스트 데이터 세트로 분리하는 것이 더 바람직하다.

>1. 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습과 테스트 데이터로 분리
>2. 1이 여의치 않다면 테스트 데이터 변환 시에는 fit()이나 fit_transform()을 적용하지 않고 학습 데이터로 이미 fit()된 Scaler 객체를 이용해 transform()으로 변환